In [1]:
from modules.scouter_handler import ScouterHandler
from modules.doc_graph_analyzer import DocGraphAnalyzer

In [2]:
keyword = "대통령"

min_rel_score = 2.0
max_length = 3000

In [3]:
scouter = ScouterHandler(size=500)

In [4]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

Query : 대통령
Scroll idx : 1 (500 docs)
Scroll idx : 2 (500 docs)
Scroll idx : 3 (500 docs)
Scroll idx : 4 (500 docs)
Scroll idx : 5 (500 docs)
Scroll idx : 6 (500 docs)
Scroll idx : 7 (67 docs)
Trim lower 70% of docs (920 docs are cut)
Total retrieved Doc # :  2146



In [5]:
doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

2128

In [10]:
## modified by sspark ##
doc_graph_anal = DocGraphAnalyzer(keyword, df_bound=(0.01, 0.80))
doc_graph_anal.extract_subtopic_info(doc_info_list, top_doc_num=5, top_keyword_num=5, hierarchy=True, cutting_level=1)
## modified by sspark ##

Doc #  : 2128
Node # : 1671
Edge # : 285782

Community 0 :  403 Keyword, (검찰/수사/조사/혐의/의혹)
Community 1 :  429 Keyword, (미국/북한/관계/문제/트럼프)
Community 2 :  621 Keyword, (정부/지난/정치/국민/선거)
Community 3 :  218 Keyword, (청와대/안보/실장/대표/국가)



In [6]:
## modified by sspark ##
doc_graph_anal = DocGraphAnalyzer(keyword, df_bound=(0.01, 0.80))
doc_graph_anal.extract_subtopic_info(doc_info_list, top_doc_num=5, top_keyword_num=5, hierarchy=True, cutting_level=2)
## modified by sspark ##

Doc #  : 2128
Node # : 1671
Edge # : 285782

Community 0/0 :   70 Keyword, (검찰/대표/정부/위원/미국)
Community 0/1 :   27 Keyword, (국가/선거/문제/주장/실장)
Community 0/2 :   87 Keyword, (조사/이날/가능/지난/이후)
Community 0/3 :   98 Keyword, (관계/청와대/관련/특사/남북)
Community 0/4 :  121 Keyword, (혐의/의원/북한/국회/소환)
Community 1/0 :  131 Keyword, (검찰/의원/조사/대표/관계)
Community 1/2 :  107 Keyword, (정부/국가/특사/관련/북한)
Community 1/3 :   53 Keyword, (자금/국회/회담/남북/지지)
Community 1/4 :  122 Keyword, (위원/청와대/회장/수수/지사)
Community 2/0 :  108 Keyword, (검찰/조사/이날/북한/의원)
Community 2/1 :  125 Keyword, (남북/소환/이명박/이후/회동)
Community 2/2 :  133 Keyword, (위원/청와대/특사/수사/실장)
Community 2/3 :   34 Keyword, (부장/대표/평가/설명/홍준표)
Community 2/4 :   57 Keyword, (정부/바른미래당/안보/혐의/관련)
Community 2/5 :  164 Keyword, (회장/대북/기념/대화/비서)
Community 3/0 :   69 Keyword, (검찰/혐의/대표/의원/위원)
Community 3/1 :   86 Keyword, (청와대/관계/조사/국가/지사)
Community 3/2 :   63 Keyword, (북한/정부/특사/회장/미국)



In [11]:
## modified by sspark ##
doc_graph_anal = DocGraphAnalyzer(keyword, df_bound=(0.01, 0.80))
doc_graph_anal.extract_subtopic_info(doc_info_list, top_doc_num=5, top_keyword_num=5, hierarchy=True, cutting_level=3)
## modified by sspark ##

Doc #  : 2128
Node # : 1671
Edge # : 285782

Community 0/0/2 :   28 Keyword, (검찰/대표/조사/청와대/위원)
Community 0/2/0 :   22 Keyword, (정부/미국/자금/다스/위원)
Community 0/2/1 :   21 Keyword, (대표/특사/소환/의원/남북)
Community 0/2/4 :   22 Keyword, (검찰/국회/문제/대화/회장)
Community 0/3/0 :   24 Keyword, (대표/소환/선거/국회/관계)
Community 0/3/2 :   30 Keyword, (검찰/수사/혐의/다스/회장)
Community 0/3/3 :   26 Keyword, (문재/대화/장관/트럼프/회동)
Community 0/4/1 :   42 Keyword, (검찰/특사/조사/대표/정부)
Community 0/4/2 :   26 Keyword, (청와대/관련/위원/한국/안보)
Community 0/4/3 :   25 Keyword, (소환/대북/정상/의혹/수수)
Community 1/0/1 :   40 Keyword, (북한/대표/지난/선거/대화)
Community 1/0/2 :   31 Keyword, (검찰/조사/특사/정부/수사)
Community 1/0/3 :   43 Keyword, (혐의/가능/회담/문제/기념)
Community 1/2/3 :   38 Keyword, (정부/조사/청와대/의원/남북)
Community 1/2/4 :   37 Keyword, (검찰/문재/관련/특사/관계)
Community 1/3/1 :   21 Keyword, (청와대/관련/조사/회장/검찰)
Community 1/4/0 :   25 Keyword, (위원/남북/관련/청와대/조사)
Community 1/4/1 :   25 Keyword, (미국/국가/선거/이후/장관)
Community 1/4/2 :   26 Keyword, (문재/소환/대변/안보/한국)
Community 1/4/3 : 

In [7]:
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type='newspaper', trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]

In [9]:
top_doc_info_list[0][0]

{'newsTitle': '親文 견제에  안희정의 큰꿈, 기로에 서다',
 'extContent': '충남지사 3선(選) 도전을 포기한 안희정〈그림〉 지사의 재 보선 출마 문제 등을 놓고 여권 내부에서 논란이 커지고 있다. 안 지사는 공식적으로는 6 13 지방선거와 함께 치러지는 국회의원 재 보선도 불출마하겠다는 입장을 보여왔다. 하지만 안 지사 주변에선  수도권 재 보선뿐 아니라 오는 8월 민주당 당대표 선거 출마도 모두 열려 있다 는 말이 나온다. 이에 친문 진영에선  문재인 대통령과 대립각을 세워온 안 지사가 너무 빨리 등판하면 국정운영에 부담을 줄 수 있다 며 난색을 표하고 있다. 안 지사는 민주당 잠룡 중 유일하게 어떤 정치 계획도 밝히지 않고 있다. 이는 또 다른 차기 주자인 이재명 성남시장이 경기지사 경선 출마를 위해 이날 성남시의회에 사임 의사를 밝힌 것과 대비된다. 이 시장은 선거법상 성남시장 사퇴 시한 전날인 오는 14일 퇴임식을 가진 뒤 경기지사 공천 경쟁에 본격적으로 뛰어들 계획이다. 박원순 서울시장 역시 3선 도전에 나선 상태다. 그런데 안 지사는 지난 대선 경선 때 문 대통령 다음으로 많은 지지를 받고도 조용히 지내고 있다. 작년 12월 말에는 기자회견을 통해 충남지사 3선 불출마를 공식화했다.  재 보선 출마도 고려하고 있지 않다 고 했다.  현재로선 이라는 단서를 붙였지만 불출마로 받아들여졌다. 이후 안 지사가 도지사 법정임기(6월 30일)를 마친 뒤 해외 유학을 떠날 것이란 얘기가 돌았다.  안 지사가 재 보선 출마 의사를 타진했지만 청와대가 반대했다 는 추측성 보도도 나왔다. 안 지사 측은  유학설, 퇴짜설 모두 사실무근 이라고 했다. 친문계 인사들은  안 지사가 당분간은 나서지 않는 게 본인이나 당, 정권 모두를 위해 좋다 고 말하고 있다. 한 민주당 의원은  안 지사가 재 보선에 출마하거나 당대표로 나서면 문 대통령에게 집중돼야 할 시선이 분산된다 며  당도 계파 분열 양상을 보일 수 있다 고 했다.  집권 2년차에 당 청 